## Decoding how the brain tracks peer drinking in groups 

<font size="3"> <span style='color: #7393B3'> Mia Jovanova & Danielle Cosme
    
<font size="3"> <span style='color: gray'> Main analysis presented at ICA, May 2022 Paris
    
   * <font size="3">[Pre-registration](https://osf.io/2qny7/)
   * [Slides](https://docs.google.com/presentation/d/1CAG-N_qar-9EMb5qb8QhGywAI5njhvCUFXiKtNYRLYM/edit#slide=id.g12f6e35eda1_0_120) 
    


### Study background

* The current study investigates how the brain represents the perceived health habits of
peers, such as alcohol use, within existing social groups. 

* We will use multivoxel pattern analysis
(MVPA) to ask the following questions: (1) First, how does the brain support the ability to take
the perspective of peers who engage in varying drinking habits (e.g. drink more vs. less than
oneself)? Second, does the brain activate information about peer drinking habits even when
passively viewing peer faces? And to what degree is this neural representation about peers
shared across individuals vs. idiosyncratic? 


* Answering these questions may provide insight into
how individuals process information about others more generally and respond to health-related
influences outside the laboratory. Moreover, this investigation provides an opportunity to bridge
health behavior theories (e.g., about social norms and social learning) with models of
the social brain to predict individual differences in susceptibility to social influence.

### Hypothesis:

   * H1a: Conducting MVPA across participants, we expect that shared patterns of activity in brain regions  will be able to distinguish trials when participants take the perspective of peers who drink more from trials when they  take the perspective of peers who drink less with above chance accuracy. 


### Method:
* We will use supervised classification to determine the extent to which we can distinguish
whether individuals were instructed to take the perspective of a peer who drinks more (“peer
high”) or less (“peer low”) than themselves, using patterns of activity in the whole brain. We will
use smoothed data (smoothing kernel = 6mm) in MNI space—that is, we will average trials to
yield separate patterns for peer high and peer low for each participant and task run (4 total). We
will exclude individual task runs that are contaminated by motion, defined using the standard
operating procedures for this project. 


* We will use a regularized logistic classifier to facilitate
interpretability of the weights with the default hyperparameters (C = 1.0) implemented in
NLTools. We will use 5-fold cross-validation to assess classification accuracy at the run level
while controlling for dependence of runs within person using stratified sampling so that
participant’s data is kept together within each cross-validation fold. Standard errors will be
averaged across cross-validation iterations. If the average accuracy across the folds is
significantly higher than 50%, we will interpret this as evidence that taking the perspective of
peers perceived to drink more or less than themselves is dissociable based on patterns of
activity within the whole-brain

----------------

### Load packages

* nltools
* numpy
* pandas
* niliearn
* glob

In [ ]:
import numpy as np
import pandas as pd
import glob
from nltools.datasets import download_collection
from nltools.data import Brain_Data
from nilearn import image
from nilearn.masking import compute_epi_mask
from nltools.analysis import Roc
from nltools.stats import correlation
import nilearn.plotting as plotting
from nilearn.masking import compute_epi_mask, apply_mask, unmask
from nilearn.image import resample_to_img

### Load files

* Neuroimaging task files
    
* Metadata: information about trials and participants
    
    - <span style='color: gray'>  'Upreg' trials when participants are prompted to think about peers who drink more
    - <span style='color: gray'> 'Downreg' trials when participants are prompted to think about peers who drink less
    
   

In [ ]:
files=sorted(glob.glob("/data00/projects/MURI/data/BIDS/derivatives/nipype/perspective/*.nii")) 
metadata=pd.read_csv("/data00/projects/MURI/scripts/jupyterhub_users/CNLab/analyses/MVPA_PPT_mj/metadata/metadata_BMIN525_development.csv")

### Prepare brain files

* Brain_data is a helpful tool to convert neuroimaging data in python as a vector rather than a 3-dimensional matrix. Source: (https://nltools.org/api.html)

In [ ]:
data = Brain_Data(files,X=metadata)

* Double check sample size: each individual (N = 34) should have 2 brain files for a total of 68 brainfiles

In [ ]:
sample_size = len(files)
if sample_size  == len(np.unique(metadata['pID']))*2:
    print("sample size:", sample_size, "is CORRECT")

if sample_size  != 68:
    print("sample size", sample_size, "is INCORRECT. STOP!")

### Set up classification 

In [ ]:
X_development = data.X.loc[data.X['sample']=='development']
Y_development = X_development['trial_type']
print('Development N data = ', len(X_development))
X_development =pd.DataFrame(X_development)
development = data[np.where(data.X['id'].isin(X_development['id']))[0]]
upreg = development[np.where(development.X['trial_type']=='upreg')[0]]
downreg = development[np.where(development.X['trial_type']=='downreg')[0]]
dat = upreg.append(downreg)
dat.Y = pd.DataFrame(np.concatenate([np.ones(len(upreg)),np.zeros(len(downreg))]))
subject_id = np.concatenate([upreg.X['pID'].values,downreg.X['pID'].values])

### Run classification

In [ ]:
logistic_stats = dat.predict(algorithm='logistic',
                                  cv_dict={'type': 'kfolds', 'n_folds': 5 , 'subject_id' : subject_id, 
                                           'stratified':dat.Y},
                                  **{'max_iter': 1000})

In [ ]:
roc_logistic = Roc(input_values=logistic_stats['prob_xval'][::,1], 
binary_outcome=logistic_stats['Y'].astype(bool), threshold_type='optimal_overall',forced_choice=None)
roc_logistic.plot()
roc_logistic.summary()

### Plot & save resulting weightmaps

In [ ]:
plotting.plot_glass_brain(logistic_stats['weight_map'].to_nifti(), display_mode='lyrz', colorbar=True, plot_abs=False, cmap=plotting.cm.ocean_hot)
logistic_stats['weight_map'].write('logistic_weightmap_decoding_peers_H1a.nii')